In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
train=pd.read_csv("C://Users//Vishal//Desktop//techgig//1//traindata.csv")

In [3]:
train['DestinationAirportID'].nunique()

1989

In [153]:
train['SourceAirportID'].nunique()

1991

Here we come to know that there is missing data in the source and destination fields

In [4]:
train['cost']=train['AirlineID'].apply(lambda x: 1)

In [5]:
d = {'SourceAirportID': [], 'DestinationAirportID':[],'cost':[]}
ans=pd.DataFrame(d)
ans

,SourceAirportID,DestinationAirportID,cost


In [7]:
test=pd.read_csv("C://Users//Vishal//Desktop//techgig//1//testdata.csv")

In [8]:
test=test.sort_values(by=['SourceAirportID','DestinationAirportID'])

In [9]:
source_test=test['SourceAirportID'].unique()
source_train=train['SourceAirportID'].unique()

Here we find the source airports which are in the test data but not in the training data

In [12]:
x=train[['SourceAirportID','DestinationAirportID','cost']]

In [26]:
x=sorted(train['DestinationAirportID'].unique())
y=sorted(train['SourceAirportID'].unique())

In [174]:
print(y[50])
print(x[49])

AFT
AFU


In [13]:
x=x.sort_values(by=['SourceAirportID','DestinationAirportID'])

In [14]:
x=x.reset_index()

In [15]:
x=x.pivot(index='SourceAirportID',columns='DestinationAirportID',values='cost')

We convert our data into a matrix of n * n, with a cost of 1 if there is a path from a to b and 0 if not 

In [16]:
x=x.reset_index()

In [17]:
x=x.fillna(0)

In [43]:
np.array(x['SourceAirportID'])

array(['ADV', 'ADW', 'ADX', ..., 'VCO', 'VCP', 'VCQ'], dtype=object)

In [23]:
for i in np.array(x['SourceAirportID']):
    if i not in x.columns:
        x[i]=x['AEE'].apply(lambda x: 0)

We add airports which are in the source column but not in the destination column

In [26]:
for i in x.columns:
    if i=="SourceAirportID":
        continue
    else:
        if i not in np.array(x['SourceAirportID']):
            x=x.append({'SourceAirportID':i},ignore_index=True)

In [27]:
x=x.fillna(0)

Now we add airports which are in the destination column but not as a source

In [28]:
x=x.sort_values(by=['SourceAirportID'])

In [29]:
y=x.drop(['SourceAirportID'],axis=1)

In [30]:
y = y.reindex(sorted(y.columns), axis=1)

In [31]:
y

DestinationAirportID,ADV,ADW,ADX,ADY,ADZ,AEA,AEB,AEC,AED,AEE,...,VCH,VCI,VCJ,VCK,VCL,VCM,VCN,VCO,VCP,VCQ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we sort the data by values in the source column and the destination by alphabetical order
This gives us our adjacency matrix of 2000 rows and 2000 columns

In [32]:
import sys
class Graph(): 
  
    def __init__(self, vertices): 
        self.V = vertices 
        self.graph = [[0 for column in range(vertices)]  
                    for row in range(vertices)] 
  
    def printSolution(self, dist): 
        print ("Vertex \tDistance from Source")
        for node in range(self.V): 
            print (node, "\t", dist[node]) 
   
    def minDistance(self, dist, sptSet): 
  
        
        min = 99999 
        min_index=0
        for v in range(self.V): 
            if dist[v] < min and sptSet[v] == False: 
                min = dist[v] 
                min_index = v 
  
        return min_index 
  
    def dijkstra(self, src): 
  
        dist = [99999] * self.V 
        dist[src] = 0
        sptSet = [False] * self.V 
  
        for cout in range(self.V):     
            u = self.minDistance(dist, sptSet)      
            sptSet[u] = True
            for v in range(self.V): 
                if (self.graph[u][v] > 0 and sptSet[v] == False and dist[v] > dist[u] + self.graph[u][v]): 
                        dist[v] = dist[u] + self.graph[u][v] 
  
        
        return(dist)
  

g = Graph(2000) 
g.graph = np.array(y); 


Above is the Dijikstra Algorithm which is used to find the minimum distance between two nodes in a graph

In [41]:
d = {'SourceAirportID': [], 'DestinationAirportID':[],'cost':[]}
ans=pd.DataFrame(d)
ans

,SourceAirportID,DestinationAirportID,cost


In [ ]:
for i in range(0,2000):
    print(i)
    dist=g.dijkstra(i)
    for j in range(0,len(dist)):
        ans=ans.append({'SourceAirportID':y.columns[i],"DestinationAirportID":y.columns[j],"cost":dist[j]},ignore_index=True)

Now we apply Dijikstra for all our 2000 nodes and find the minimum distance between them and append it to our solution with source,destination and cost

In [55]:
def compute_cost(d):
    if d>90000: #no path
        return int(-1) 
    else: #hop calculation
        return int(d-1)

In [45]:
ans=ans[ans['SourceAirportID']!=ans['DestinationAirportID']] #remove same source same destination entries
ans['cost']=ans['cost'].apply(compute_cost)

C:\Users\Vishal\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
test

,PairID,SourceAirportID,DestinationAirportID
1999,1804299539,ADV,ADW
3998,1804309438,ADV,ADX
5997,1804319598,ADV,ADY
7996,1804329615,ADV,ADZ
9995,1804339639,ADV,AEA
...,...,...,...
3988004,1824240553,VCQ,VCL
3990003,1824250695,VCQ,VCM
3992002,1824260688,VCQ,VCN
3994001,1824270785,VCQ,VCO


In [56]:
ans['Hops']=ans['cost'].apply(compute_cost)

In [63]:
ans=ans.sort_values(by=['SourceAirportID','DestinationAirportID'])

we sort the values by sourceid and destinationid

In [87]:
hop=ans['Hops']

In [107]:
pair=test['PairID']

In [117]:
pr=[]
for i in pair:
    pr.append(i)

In [126]:
hp=[]
for i in hop:
    hp.append(i)

In [128]:
hops=pd.DataFrame(hp,columns=['Hops'])
pairs=pd.DataFrame(pr,columns=['PairID'])

In [150]:
last=pd.concat([pairs,hops],axis=1)

Now we combine the pairid and the hops into a dataframe which is last 

In [151]:
last.to_csv(r'C:/Users/Vishal/Desktop/ans/last1.csv',index=False)

We convert the dataframe into csv file and submit it